In [1]:
import pandas as pd
stock_list = pd.read_csv("../database/yearly/tw_stock_list.csv")
s = stock_list.sort_values(by="volume", ascending=False)
s.head()

,Unnamed: 0,ticker,codename,price,volume,volume_ntd,ticker.1,codename.1,market,listed,TW_TWO
1827,2095,1314,中石化,11.25,197173,22.182,1314,中石化,1314.TW,True,TW
1115,1213,2618,長榮航,35.55,146037,51.916,2618,長榮航,2618.TW,True,TW
1646,1870,2515,中工,17.70,129256,22.878,2515,中工,2515.TW,True,TW
1058,1133,2501,國建,38.30,128465,49.202,2501,國建,2501.TW,True,TW
706,720,2363,矽統,61.80,124192,76.751,2363,矽統,2363.TW,True,TW


In [35]:
from functions.functions import data_strat1, nav_returns_backtest_only
from datetime import datetime, timedelta
strat = "strat2"
stock_list = pd.read_csv("../database/yearly/tw_stock_list.csv")
for start_year in [2020, 2021, 2022, 2023]:
    start = f"{start_year}-01-01"
    end = f'{start_year+1}-01-01'
    end=datetime.now()
    returns = []
    winrates = []
    avg_ret = []
    trade_total = []
    last_open_date = []
    last_open_price = []
    tickers = []
    df_final = pd.DataFrame()
    for ticker in s['market'][:500]:
        ticker_bar = ticker.replace(".", "_")
        df = data_strat1(ticker_bar, start, end)

        return_total, winrate, avg_return, trades, last_openposition_date, last_openposition_price = nav_returns_backtest_only(df)
        returns.append(return_total)
        winrates.append(winrate)
        avg_ret.append(avg_return)
        trade_total.append(trades)
        tickers.append(ticker[:4])
        # last_open_date.append(df.index[last_openposition_date])
        last_open_price.append(last_openposition_price)
    df_final['ticker'] = tickers
    df_final['return_total'] = returns
    df_final['winrate'] = winrates
    df_final['avg_return'] = avg_ret
    df_final['trade_total'] = trade_total
    # df_final['last_openposition_date'] = last_open_date
    df_final['last_openposition_price'] = last_open_price
    df_final.to_csv(f"backtest_by_year/{strat}_{start_year}_{start_year+1}.csv")

c:\Users\cwcha\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\cwcha\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\cwcha\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\cwcha\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\cwcha\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\cwcha\AppData\Loca

In [36]:
start = 2020
df_final = pd.read_csv(f"backtest_by_year/strat1_{start}_{start+1}.csv")
df_outsample = pd.read_csv(f"backtest_by_year/strat1_{start+1}_{start+2}.csv")
return_1 = list(df_final[df_final['return_total'] > 0.1]['ticker'])
return_2 = list(df_final[df_final['return_total'] > 0.2]['ticker'])
return_3 = list(df_final[df_final['return_total'] > 0.3]['ticker'])
return_4 = list(df_final[df_final['return_total'] > 0.4]['ticker'])
return_5 = list(df_final[df_final['return_total'] > 0.5]['ticker'])
len(return_1), len(return_2), len(return_3), len(return_4), len(return_5)

(152, 82, 44, 22, 14)

In [50]:
import numpy as np
max_returns = []
min_returns = []
starts = []
return_thresholds = []
stock_list_len = []
mean_returns = []
trade_totals = []
mean_winrates = []
final = pd.DataFrame()
for start in [2020, 2021, 2022, 2023]:
    df_final = pd.read_csv(f"backtest_by_year/strat1_{start}_{start+1}.csv")
    df_outsample = pd.read_csv(f"backtest_by_year/strat1_{start+1}_{start+2}.csv")
    for return_threshold in [0.1, 0.2, 0.3, 0.4, 0.5]:
        return_1 = list(df_final[df_final['return_total'] > return_threshold]['ticker'])

        return_5_avg = df_outsample[df_outsample['ticker'].isin(return_1)]

        max_return = np.max(return_5_avg['return_total'])
        min_return = np.min(return_5_avg['return_total'])
        mean_return = np.mean(return_5_avg['return_total'])
        mean_winrate = np.mean(return_5_avg['winrate'])
        trade_total = np.sum(return_5_avg['trade_total'])
        max_returns.append(max_return)
        min_returns.append(min_return)
        return_thresholds.append(return_threshold)
        starts.append(start)
        stock_list_len.append(len(return_1))
        mean_returns.append(mean_return)
        trade_totals.append(trade_total)
        mean_winrates.append(mean_winrate)
final['starts'] = starts
final['max_returns'] = max_returns
final['min_returns'] = min_returns
final["return_thresholds"] = return_thresholds
final["stock_list_len"] = stock_list_len
final["mean_returns"] = mean_returns
final["trade_totals"] = trade_totals
final["mean_winrates"] = mean_winrates
final.sort_values(by='mean_returns', ascending=False)
final.sort_values(by="return_thresholds", ascending=False)

,starts,max_returns,min_returns,return_thresholds,stock_list_len,mean_returns,trade_totals,mean_winrates
19,2023,0.062076,-0.138331,0.5,14,-0.026993,13,0.071429
4,2020,1.089247,-0.400875,0.5,12,0.145956,56,0.544048
9,2021,0.306213,-0.337636,0.5,13,-0.035828,65,0.404945
14,2022,0.320914,-0.035978,0.5,6,0.135564,27,0.661111
3,2020,1.089247,-0.400875,0.4,20,0.124919,84,0.529762
18,2023,0.264267,-0.237738,0.4,22,-0.012399,22,0.227273
8,2021,0.306213,-0.426227,0.4,28,-0.045748,139,0.433248
13,2022,0.320914,-0.081633,0.4,8,0.107300,37,0.683333
2,2020,1.089247,-0.400875,0.3,32,0.107157,128,0.490997
7,2021,0.306213,-0.426227,0.3,54,-0.070697,252,0.428660


In [47]:
max_returns = []
min_returns = []
starts = []
return_thresholds = []
stock_list_len = []
mean_returns = []
trade_totals = []
mean_winrates = []
final_negative = pd.DataFrame()
for start in [2020, 2021, 2022, 2023]:
    df_final = pd.read_csv(f"backtest_by_year/strat1_{start}_{start+1}.csv")
    df_outsample = pd.read_csv(f"backtest_by_year/strat1_{start+1}_{start+2}.csv")
    for return_threshold in [-0.1, -0.2, -0.3, -0.4, -0.5]:
        return_1 = list(df_final[df_final['return_total'] < return_threshold]['ticker'])

        return_5_avg = df_outsample[df_outsample['ticker'].isin(return_1)]
        
        max_return = np.max(return_5_avg['return_total'])
        min_return = np.min(return_5_avg['return_total'])
        mean_return = np.mean(return_5_avg['return_total'])
        mean_winrate = np.mean(return_5_avg['winrate'])
        trade_total = np.sum(return_5_avg['trade_total'])
        max_returns.append(max_return)
        min_returns.append(min_return)
        return_thresholds.append(return_threshold)
        starts.append(start)
        stock_list_len.append(len(return_1))
        mean_returns.append(mean_return)
        trade_totals.append(trade_total)
        mean_winrates.append(mean_winrate)
final_negative['starts'] = starts
final_negative['max_returns'] = max_returns
final_negative['min_returns'] = min_returns
final_negative["return_thresholds"] = return_thresholds
final_negative["stock_list_len"] = stock_list_len
final_negative["mean_returns"] = mean_returns
final_negative["trade_totals"] = trade_totals
final_negative["mean_winrates"] = mean_winrates
final_negative.sort_values(by='mean_returns', ascending=False)

,starts,max_returns,min_returns,return_thresholds,stock_list_len,mean_returns,trade_totals,mean_winrates
4,2020,0.394371,0.050820,-0.5,2,0.222596,8,0.466667
0,2020,0.999017,-0.383713,-0.1,150,0.089990,623,0.575016
10,2022,0.764276,-0.270476,-0.1,162,0.082039,684,0.523883
1,2020,0.999017,-0.383713,-0.2,73,0.080490,304,0.532257
2,2020,0.775098,-0.307946,-0.3,26,0.077249,123,0.497619
11,2022,0.637127,-0.270476,-0.2,70,0.070379,317,0.545986
14,2022,0.192406,-0.082850,-0.5,2,0.054778,6,0.750000
12,2022,0.617074,-0.156180,-0.3,25,0.049096,111,0.536000
15,2023,0.477736,-0.139573,-0.1,34,0.048495,41,0.411765
16,2023,0.130816,-0.093486,-0.2,5,0.026445,3,0.400000


In [45]:
return_5_avg['return_total']

Series([], Name: return_total, dtype: float64)